## Astrometry.net solutions - 15-Jul-25

Craig Lage - Attempting to find a.net solutions for crowded fields of 3I

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.geom import Point2D, SpherePoint, Angle, AngleUnit
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
import astropy.wcs
from astropy.io import fits
import pandas as pd
import pickle as pkl
from lsst.summit.utils.plotting import plot
from lsst.daf.butler import Butler
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
# Code originally from Merlin Fisher-Levine to solve StarTracker images
def runCharacterizeImage(exp, thresh, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
        Here we have modified it to be a numeric threshold
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = True
    charConfig.doDeblend = True
    charConfig.doNormalizedCalibration = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = thresh
    charConfig.detection.thresholdType = 'value'
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult



In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
csv_filename = "/home/c/cslage/u/LSSTCam/data/3I_better_visits_radecs_forCraig.csv"
csv_data = pd.read_csv(csv_filename)

ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

# a.net parameters:

image_width = 4072
image_height = 4000
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = 0.20
scale_err = 2.0
radius = 1.0


# The cell belows tries to solve for the location of 3I given Colin's location.
## It needs to be run for each detector, but the only detections were in detectors 30, 31 and 3.

In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/3I_Pixel_Locations_det30_16Jul25.txt"
outfile = open(outfilename, 'w')
outfile.write(f"expId \t detector \t pixelX \t pixelY \t inChip?\n")


detector = 30
for expId in range(2025071200517, 2025071200593):
    try:
        thisRow = csv_data[csv_data['visit'] == expId]
        target_ra = thisRow['ra_horizons'].values[0]
        target_dec = thisRow['dec_horizons'].values[0]
        calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
        raw = butler.get('raw', detector=detector, exposure=expId)
        raw_wcs = raw.getWcs()
        center_ra = raw_wcs.pixelToSky(Point2D(2036, 2000)).getRa().asDegrees()
        center_dec = raw_wcs.pixelToSky(Point2D(2036, 2000)).getDec().asDegrees()
        charResult = runCharacterizeImage(calexp, 10000, 200)
        
        # Cut down to the brightest sources and eliminate close pairs
        sourceCatalog = charResult.sourceCat
        maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
        selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
        brightestSource = sourceCatalog.subset(selectBrightestSource)
        brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                             brightestSource['base_SdssCentroid_y'][0])
        brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.1)
        #print(f"expId:{expId}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
        #print(f"Brightest centroid at {brightestCentroid}")
        cat = brightCatalog
        
        sources = sourceCatalog.asAstropy()
        sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
        print(len(sources))
        
        # Eliminate close pairs
        close_limit = 50 # pixels
        for source1 in sources:
            x1 = source1['base_SdssCentroid_x']
            y1 = source1['base_SdssCentroid_y']
            for source2 in sources:
                x2 = source2['base_SdssCentroid_x']
                y2 = source2['base_SdssCentroid_y']
                if np.sqrt((x1-x2)**2 + (y1-y2)**2) < close_limit:
                    sources.remove_rows([source2.index])
                  
        print(f"{expId} has {len(sources)} sources to feed to a.net")
         # Get a.net solution    
        wcs_header = ast.solve_from_source_list(sources['base_SdssCentroid_x'], sources['base_SdssCentroid_y'],
                                                image_width, image_height, scale_units=scale_units,
                                                scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                                center_ra=center_ra, center_dec=center_dec, radius=radius,
                                                solve_timeout=120, crpix_center=True)
    
        print(f"{expId} a.net solution {wcs_header['CRVAL1']},{wcs_header['CRVAL2']}")
            
    
        wcs_from_fits = astropy.wcs.WCS(wcs_header)
        skyLocation = SkyCoord(target_ra*u.deg, target_dec*u.deg)
        pixel_location = wcs_from_fits.world_to_pixel(skyLocation)
        pixX = float(pixel_location[0])
        pixY = float(pixel_location[1])
        if pixX<0 or pixX>4072 or pixY<0 or pixY>4000:
            outfile.write(f"{expId} \t {detector} \t {pixX:.2f} \t {pixY:.2f} \t No\n")
        else:
            outfile.write(f"{expId} \t {detector} \t {pixX:.2f} \t {pixY:.2f} \t Yes\n")
        print(expId, pixX, pixY)
    except:
        print(f"{expId} failed!")
        outfile.write(f"{expId} Failed!! \n")

outfile.close()
    

# The cell below creates a dictionary of WCS objects.

In [ ]:
wcsDict = {}
for detector in [3,30,31]:
    filename = f"/home/c/cslage/u/LSSTCam/data/3I_Pixel_Locations_det{detector:02d}_16Jul25.txt"
    data = np.loadtxt(filename, skiprows=1, dtype='str')
    for n in range(len(data)):
        expId = int(data[n][0])
        key = f"{expId}_{detector:02d}"
        try:
            thisRow = csv_data[csv_data['visit'] == expId]
            target_ra = thisRow['ra_horizons'].values[0]
            target_dec = thisRow['dec_horizons'].values[0]
            calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
            raw = butler.get('raw', detector=detector, exposure=expId)
            raw_wcs = raw.getWcs()
            center_ra = raw_wcs.pixelToSky(Point2D(2036, 2000)).getRa().asDegrees()
            center_dec = raw_wcs.pixelToSky(Point2D(2036, 2000)).getDec().asDegrees()
            charResult = runCharacterizeImage(calexp, 10000, 200)
            
            # Cut down to the brightest sources and eliminate close pairs
            sourceCatalog = charResult.sourceCat
            maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
            selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
            brightestSource = sourceCatalog.subset(selectBrightestSource)
            brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                                 brightestSource['base_SdssCentroid_y'][0])
            brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.1)
            #print(f"expId:{expId}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
            #print(f"Brightest centroid at {brightestCentroid}")
            cat = brightCatalog
            
            sources = sourceCatalog.asAstropy()
            sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
            print(len(sources))
            
            # Eliminate close pairs
            close_limit = 50 # pixels
            for source1 in sources:
                x1 = source1['base_SdssCentroid_x']
                y1 = source1['base_SdssCentroid_y']
                for source2 in sources:
                    x2 = source2['base_SdssCentroid_x']
                    y2 = source2['base_SdssCentroid_y']
                    if np.sqrt((x1-x2)**2 + (y1-y2)**2) < close_limit:
                        sources.remove_rows([source2.index])
                      
            print(f"{expId} has {len(sources)} sources to feed to a.net")
             # Get a.net solution    
            wcs_header = ast.solve_from_source_list(sources['base_SdssCentroid_x'], sources['base_SdssCentroid_y'],
                                                    image_width, image_height, scale_units=scale_units,
                                                    scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                                    center_ra=center_ra, center_dec=center_dec, radius=radius,
                                                    solve_timeout=120, crpix_center=True)
        
            print(f"{expId} a.net solution {wcs_header['CRVAL1']},{wcs_header['CRVAL2']}")
                
        
            wcs_from_fits = astropy.wcs.WCS(wcs_header)
            wcsDict[key] = [wcs_header, wcs_from_fits]
            print(f"Done with {key}")
        except:
            continue
filename = "/home/c/cslage/u/MTMount/mount_plots/wcsDict_3I.pkl"
with open(filename, 'wb') as f:
    pkl.dump(wcsDict, f)
    